In [1]:
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mode 
from matplotlib import cm as cm

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

    100% |████████████████████████████████| 993kB 20.1MB/s 


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1GXhb9LJJshv_gFdiMS6PujIG8SlhaZqv' # The shareable link
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('properatti.csv')  

In [0]:
#Opcional para leer columnas completas
#pd.set_option('display.max_colwidth', -1)

In [0]:
df = pd.read_csv("properatti.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [0]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [0]:
#Faltan varios datos importantes. Los de ubicación no tanto en principio, pero precio y superficie.
total = 121220
faltantes=df.isnull().sum()
print ('Datos faltantes')
faltantes

Datos faltantes


Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [0]:
print ('Proporción de datos faltantes')
faltantes/total*100

Proporción de datos faltantes


Unnamed: 0                     0.000000
operation                      0.000000
property_type                  0.000000
place_name                     0.018974
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   15.440521
lat-lon                       42.525986
lat                           42.525986
lon                           42.525986
price                         16.837156
currency                      16.837981
price_aprox_local_currency    16.837156
price_aprox_usd               16.837156
surface_total_in_m2           32.443491
surface_covered_in_m2         16.422208
price_usd_per_m2              43.394654
price_per_m2                  27.686850
floor                         93.483749
rooms                         60.905791
expenses                      88.234615
properati_url                  0.000000
description                    0.001650
title                          0.000000


In [0]:
#Top 10 lugares, están muy desgranados
#Medio que en place name no hay criterio
df['place_name'].value_counts().head(10)

Córdoba          9254
Rosario          8504
Mar del Plata    6534
Tigre            3324
Nordelta         3315
Belgrano         2992
Palermo          2885
La Plata         2534
Caballito        2273
Pilar            2228
Name: place_name, dtype: int64

In [0]:
#Solo hay operaciones de venta
df['operation'].value_counts()

sell    121220
Name: operation, dtype: int64

In [0]:
#Es cualquiera el rango de pisos se puede repararar parcialmente con regex
print ('Piso mínimo es:' + str(df['floor'].min())) 
print ('Piso máximo es:' + str(df['floor'].max()))
df['floor'].value_counts().head()

Piso mínimo es:1.0
Piso máximo es:3150.0


1.0    2080
2.0    1542
3.0     914
4.0     658
5.0     484
Name: floor, dtype: int64

In [0]:
#Rango de expenses.
#No muy confiable
print ('Expensa mínima es:' + str(df['expenses'].min())) 
print ('Expensa máxima es:' + str(df['expenses'].max()))
df['expenses'].value_counts().head()

Expensa mínima es:1.0
Expensa máxima es:10001500.0


1.0       1028
1000.0     627
1500.0     617
2000.0     561
1200.0     508
Name: expenses, dtype: int64

In [0]:
#Es cualquiera el rango de AMBIENTES.
#No muy confiable
print ('Ambiente mínimo es:' + str(df['rooms'].min())) 
print ('Ambiente máximo es:' + str(df['rooms'].max()))
df['rooms'].value_counts().head()

Ambiente mínimo es:1.0
Ambiente máximo es:32.0


3.0    12567
2.0    12433
4.0     7937
1.0     7377
5.0     3782
Name: rooms, dtype: int64

In [0]:
#Patrones para ambientes y pisos con los que se pueden rescatar valores para esas columnas antes de tirarlas
#Las descripciones son IMPOSIBLES pero algo se saca, quizás sea mucho trabajo.
#Piso
#pattern= r'\|([\w.\s]*)\|([\w.\s]*)\|([\w.\s]*)'
#regex=re.compile(pattern, flags=re.IGNORECASE)
#m=regex.findall(df2['place_with_parent_names'][1])
#m

#ambiente
#(\d+\sambiente)

In [0]:
#State name parece una columna menos dispersa que place_name
#Las zonas en que está dividido BS AS son muchas pero son categorías de venta usadas por inmobiliarias
df['state_name'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [0]:
#Sugerencia
#Quedarnos con
#'property_type', 'place_name', 'state_name', 
#'geonames_id','lat-lon', 'lat', 'lon', 
#'price_aprox_usd', 'surface_total_in_m2','price_usd_per_m2',
#'floor', 'rooms' 
#Y pasarlas a español o algo con menos guiones

In [0]:
#Sacar precio promedio por state_name y llenar los faltantes de precio?
#Acá una evaluación sobre una columna con todos los precios equiparados
#Habría que calcular la cotización del dólar en el df, actualizarlo y completar para todos los que estén en pesos
#Completar los nan con promedio de precio por zona (por place sería mejor, pero es un descalabro esa columna)
df[['price_aprox_usd','state_name']].dropna().groupby('state_name').describe()

price_aprox_usd                               \
                                       count           mean           std   
state_name                                                                  
Bs.As. G.B.A. Zona Norte             24058.0  324317.123170  3.772690e+05   
Bs.As. G.B.A. Zona Oeste              7333.0  167545.648499  2.606546e+05   
Bs.As. G.B.A. Zona Sur               12085.0  198555.715572  2.260052e+05   
Buenos Aires Costa Atlántica          9059.0  143430.988472  1.470750e+05   
Buenos Aires Interior                 1814.0  183249.996582  1.983884e+05   
Capital Federal                      29018.0  269603.182879  4.575790e+05   
Catamarca                               26.0  725471.707308  5.631223e+05   
Chaco                                   36.0  178583.551389  1.355746e+05   
Chubut                                 164.0  342338.181524  6.230407e+05   
Corrientes                             470.0  206896.786681  1.864837e+05   
Córdoba                               9038.0  203624.244417  5.340228e+05   
Entre Ríos                             130.0  148113.819692  1.910945e+05   
Formosa                                 29.0   63463.367586  3.164511e+04   
Jujuy                                   10.0  301584.442000  3.208568e+05   
La Pampa                                36.0  111053.792222  7.631044e+04   
La Rioja                                 2.0   60954.385000  1.690605e+04   
Mendoza                                476.0  249705.485588  1.068573e+06   
Misiones                               348.0  121241.552299  1.298505e+05   
Neuquén                                324.0  219429.090401  3.296495e+05   
Río Negro                              361.0  300126.180693  8.746507e+05   
Salta                                  102.0  312755.743529  6.657355e+05   
San Juan                                 4.0  299575.820000  2.816914e+05   
San Luis                               208.0  282242.500433  5.167827e+05   
Santa Cruz                              19.0  176129.507895  1.728795e+05   
Santa Fe                              5413.0  139609.617118  2.937185e+05   
Santiago Del Estero                      3.0  163428.393333  1.256795e+05   
Tierra Del Fuego                        31.0  180246.755161  1.489176e+05   
Tucumán                                213.0  158878.733192  3.366362e+05   

                                                                               \
                                   min          25%         50%           75%   
state_name                                                                      
Bs.As. G.B.A. Zona Norte       5047.53  136200.0000  230000.000  3.800000e+05   
Bs.As. G.B.A. Zona Oeste       5000.00   75000.0000  110000.000  1.849600e+05   
Bs.As. G.B.A. Zona Sur         6169.20   89733.8800  140000.000  2.380000e+05   
Buenos Aires Costa Atlántica   4952.45   70000.0000  108000.000  1.700000e+05   
Buenos Aires Interior          6231.51   84125.5100  135000.000  2.200000e+05   
Capital Federal                5103.61  105000.0000  157261.000  2.850000e+05   
Catamarca                     72908.77  152500.0000  610000.000  1.190000e+06   
Chaco                         28041.83   88566.1450  140209.190  2.049438e+05   
Chubut                         8412.55  127250.0000  200000.000  3.250000e+05   
Corrientes                    32000.00  107775.0000  152650.000  2.437500e+05   
Córdoba                           0.00   70104.5900   97999.500  1.682510e+05   
Entre Ríos                    25237.65   56083.6700   95342.250  1.500000e+05   
Formosa                       49953.50   49953.5000   49953.500  5.813106e+04   
Jujuy                         47671.12   86929.6950  139648.350  6.038128e+05   
La Pampa                      28041.83   58319.8725   92538.065  1.337500e+05   
La Rioja                      49000.00   54977.1925   60954.385  6.693158e+04   
Mendoza                        8973.38   79979.8075  124500.000  2.200000e+05   
Misiones             

In [14]:
pd.set_option('display.max_columns', 50)
df[['surface_total_in_m2','surface_covered_in_m2']].fillna(0)
len(df[df.loc[:,'surface_total_in_m2']<df.loc[:,'surface_covered_in_m2']])

1106

In [38]:
df[['surface_total_in_m2','surface_covered_in_m2']].fillna(0)
valores_iguales = df[df.loc[:,'surface_total_in_m2']==df.loc[:,'surface_covered_in_m2']]
print(len(valores_iguales))
print(len(valores_iguales)/len(df.loc[:,'surface_total_in_m2']))

24173
0.19941428807127537


In [36]:
len(df[df.duplicated('description') == True])

17164

In [40]:
type(df['surface_total_in_m2'])

pandas.core.series.Series